In [ ]:
import os
main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(main_dir)

import pyvdirs.dirs as dirs
import sys
sys.path.insert(0, dirs.SYSTEM_HOME)
sys.path.insert(0, os.path.join(dirs.SYSTEM_HOME, "ToyExample"))

import numpy as np
import torch
import matplotlib.pyplot as plt

import ToyExample.toy_example as toy

%load_ext autoreload
%autoreload 2

In [36]:
ref_loss = np.ones(15**2)
learner_loss = np.arange(15**2)

In [37]:
learner_loss = torch.Tensor(learner_loss).cpu()
ref_loss = torch.Tensor(ref_loss).cpu()

In [38]:
filter_ratio = 0.5
n = 16
learnability = True

In [39]:
# Define size of super-batch
B = int(learner_loss.numel()) # Size B of a super-batch

# Each mini-batch of size b will be split in n chunks
b_over_n = int(B * (1 - filter_ratio) / n) # Size b/n of each mini-batch chunk

# Construct the scores matrix
learner_loss = learner_loss.reshape((B,1))
ref_loss = ref_loss.reshape((1,B))
scores = learner_loss - ref_loss if learnability else - ref_loss  # Shape (B,B)
device = scores.get_device()
# Rows use different learner loss ==> i is the learner/student's index
# Columns use different reference loss ==> j is the reference/teacher's index

In [30]:
B

225

In [31]:
b_over_n

7

In [50]:
b = b_over_n * n
print(b)

112


In [34]:
learner_loss.shape

torch.Size([225, 1])

In [40]:
scores

tensor([[ -1.,  -1.,  -1.,  ...,  -1.,  -1.,  -1.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
        ...,
        [221., 221., 221.,  ..., 221., 221., 221.],
        [222., 222., 222.,  ..., 222., 222., 222.],
        [223., 223., 223.,  ..., 223., 223., 223.]])

In [ ]:
# Extract basic score for each element of the super-batch
logits_ii = torch.diag(scores) # Elements from the diagonal of the scores matrix
#Q: Is this associated to the probability of picking learner i and ref j?

# Draw the first mini-batch chunk using a uniform probability distribution
indices = np.random.choice(B, b_over_n, replace=False)

In [46]:
print(logits_ii[:10])
print(logits_ii.shape)

tensor([-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.])
torch.Size([225])


In [47]:
indices

array([167,  66,  41,  12,  71, 197, 172])

In [52]:
# Get a binary mask that indicates which samples have been selected so far
is_sampled = torch.eye(B).cpu()[indices].sum(axis=0) # (B,)

In [55]:
torch.all(is_sampled[indices] == 1)

tensor(True)

In [59]:
all([element==0 for index, element in enumerate(is_sampled) if index not in indices])

True

In [ ]:
# Mask scores to only keep learner rows k that have already been sampled
logits_kj = (scores * is_sampled.view(B, 1))
print("Example row masked with 1", logits_kj[indices[0]])
print("Test", all(logits_kj[indices[0]]!=0))
for i in range(B):
    if i not in indices: 
        print("Example row masked with 0", logits_kj[i])
        print("Test", all(logits_kj[i]==0))
        break
logits_kj = logits_kj.sum(axis=0) # Sum over columns (B,)
print(logits_kj)
#Q: Associated to prob of picking learner i and an ref k that was already selected?

Example row masked with 1 tensor([166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166., 166.,
        166., 

In [ ]:
# Mask scores to only keep learner rows k that have already been sampled
logits_kj = (scores * is_sampled.view(B, 1)).sum(axis=0) # Sum over columns (B,)
#Q: Associated to prob of picking learner i and an ref k that was already selected?

# Mask scores to only keep ref columns k that have already been sampled
logits_ik = (scores * is_sampled.view(1, B)).sum(axis=1) # Sum over rows (B,)
#Q: Associated to prob of picking learner i and an ref k that was already selected?